<img src=images/xd-logo.png align=right width=300px>

# FastAPI Responses

This notebook covers how to use Pydantic to validate API responses, and how to use different responde types with FastAPI.

## Validating responses

In the same way that you can use Pydantic models to perform data validation on the API inputs, you can also use Pydantic models to validate and filter the API response outputs.

Let's create a file called `responsive_api.py` with the following POST endpoint and start a server with uvicorn that exposes the API.

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from datetime import datetime
from typing import List

app = FastAPI()

class User(BaseModel):
    id: int
    name: str = "John Doe"
    signup_ts: datetime | None = None
    friend_ids: List[int] | None = None
    password: str

@app.post("/user")
async def read_user(user: User) -> User:
    return user

And now, instead of using a web browser to access the API, you can run the following cell to use the `requests` library to generate and send a POST request.

In [ ]:
import requests

r = requests.post("http://127.0.0.1:8000/user",
                  # Notice that `id` is being passed as a string that get's casted into an integer
                  json={"id":"42", "name":"Joe","friend_ids":[110, 31], "password":"joms"})

r.json()

You can mak sure that your API resturns the data you expect it to return by using a Pydantic model as the output type of an endpoint. If your program doesn't generate valid output, Pydantic will catch it and prevent your API from returning invalid data.

You can see this happening by adding the following endpoint and sending a valid request.

In [ ]:
@app.post("/failing_user")
async def read_user(user: User) -> User:
    # the program generates invalid data, i.e. missing a password
    return {"id":user.id, "name":"John Doe"}

In [ ]:
r = requests.post("http://127.0.0.1:8000/failing_user",
                  json={"id":"42", "name":"Joe","friend_ids":[110, 31], "password":"joms"})

r.ok, r.reason

## Filtering responses

Additionally, you can also leverage the FastAPI+Pydantic combo to make sure your APIs **only** return the data you expect them to return. If the output type of an endpoint is a Pydantic model and the endpoint function attempts to return extra fields not present in the model, the response will be filtered.

Let's say what you wan't to define an entrypoint that accepts requests from users, but returns a response that doesn't include their sign-up timestamps nor their passwords. You can add the following code to `responsive_api.py`.

In [ ]:
class SafeUser(BaseModel):
    id: int
    name: str = "John Doe"
    friend_ids: List[int] | None = None


@app.post("/safe_user")
async def read_user(user: User) -> SafeUser:
    return user

And call the new endpoint with:

In [ ]:
r = requests.post("http://127.0.0.1:8000/safe_user",
                  # Notice that the query doesn't include a name so the API returns the default value
                  json={"id":"42","friend_ids":[110, 31], "password":"joms"})

r.json()

This works fine, but there's a bit of code duplication going on since you have to define multiple objects with the same fields. Can you think of a way of abstracting the common fields away?

<details>
    
  <summary><span style="color:blue">Hint</span></summary>
  
Define a Pydantic class that contains the common fields, and inherit from it to define a second class that adds the additional fields.

</details>

## Other response types

So far you've only created APIs that return JSON objects (i.e. `fastapi.responses.JSONRersponse`), which is by far the most common format to exchange data between APIs. However, there's a multitude of other data formats or response types that you might want to return.

You can investigate the following common cases:

- Returning a list of Pydantic models instead of a single model and/or returning nested Pydantic models.
- `fastapi.responses.RedirectResponse`
- Using ORJSON and what advantages it might have **(!)**.
- `fastapi.responses.StreamingResponse`
- Returning XML responses.
- `fastapi.responses.FileResponse`
- Manual exception handling (e.g. a user is not found) with `fastapi.HTTPException`.
- Constructing custom response types by inhereting from the `Response` class.

## Summary

If you use Pydantic types to model the output of you APIs:
  - You can validate the output of your endpoints. If the data is invalid you'd want your code to fail, not to return invalid data.
  - It allows you to declare what data your API returns (and filter what it shouldn't return) in an ergonomic way.
  - There is a single *source of truth* about what data is returned: the Pydantic models.
  - You get documentation *for free* and an automatic JSON schema (at `entrypoint/openapi.json`, also accesible from the `/docs` endpoint).

And moreoever, FastAPI supports a bunch of additional response types not limited to raw-ish data formats.

### Miscellaneous tips

> If you need to return an object that can't be a Pydantic model for some reason (e.g. database object, raw dictionary), you can still do data validation and filtering by specifying the output type of your endpoint to be a Pydantic model. However, this will break linting and other IDE niceties. To circumvent this you can set the `response_model=PydanticModel` parameter in the endpoint decorator and specifying the return type of your endpoint function as `Any`.

> You can disable the response validation that FastAPI performs by default by setting the endpoitn argument `response_model=None`, but don't unless you have a good reason.

> You can use the `default_response` argument of the `FastAPI` constructor to specify a default response type for all endpoints. Quite common to use with `default_response_class=ORJSONResponse` if you're building a pure JSON API. 